In [ ]:
import flopy
import os
import geopandas as gpd
import basic
import contextily as ctx
import pandas as pd
import numpy as np
import sys
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import conda_scripts.utils as utils


from flopy.utils.postprocessing import (
    get_transmissivities,
    get_water_table,
    get_gradients,
)
import flopy.utils.binaryfile as bf


In [ ]:
import postprocess
import write_inflows
import Hydrographs
import post_process_heads

In [ ]:

run = "June2012"
# Hydrographs.run(run_name=run, reload = True)
# postprocess.run(run)

post_process_heads.run(run_name=run, head_frequency=5)

In [ ]:
import basic

In [ ]:
info, swr_info, sfr_info, riv_keys_info = basic.load_params('June2012')

datestart = info['start_date']

name = info['name']

out_folder = basic.out_folder('June2012')

basic
print(datestart)
print(out_folder)

In [ ]:
def set_bounds(ax, locname):
    
    locs =   { "MIRABEL": [-122.909128, 38.4911643, -122.85212, 38.52827]}
    tb = locs[locname.upper()]
    box = [tb[0], tb[2], tb[1], tb[3]]

    ax.set_extent(box)
    

In [ ]:
m = basic.load_model()

In [ ]:


hdsobj = bf.HeadFile(os.path.join(m.model_ws, 'Results','RRbin.hds'))
hds = hdsobj.get_data(kstpkper=hdsobj.get_kstpkper()[-1])
hds.shape




Plot heads in each layer; export the heads and head contours for viewing in a GIS

for more information about GIS export, type help(export_array), for example


In [ ]:
# swr_reaches = basic.get_swr_reaches(m )
swr_reaches = gpd.read_file("GIS/SWR_Reaches.shp")

In [ ]:

hdsobj.get_data(totim=)

In [ ]:
head = ''' <html>
<head>
<style>
div.gallery {
  margin: 5px;
  border: 1px solid #ccc;
  float: left;
  width: 180px;
}

div.gallery:hover {
  border: 1px solid #777;
}

div.gallery img {
  width: 100%;
  height: auto;
}

div.desc {
  padding: 15px;
  text-align: center;
}
</style>
</head>
<body>'''

tail = '''</body>
</html> '''

In [ ]:
with open(os.path.join(out_folder, 'gallery_wl_new.html'), 'w') as outfile:
    outfile.write(head)
    for v in gallery:
        outfile.write(v)
    outfile.write(tail)

In [ ]:

times = hdsobj.get_times()[::5]
gallery = list()

fig, axes = plt.subplots(1, 3, figsize=(20, 8.5), subplot_kw=dict(projection=ccrs.epsg(2226)))
axes = axes.flat
grid = m.modelgrid
for cnt,time in enumerate(times):
    hds = hdsobj.get_data(totim = time)
    
    date = pd.to_datetime(datestart) + pd.to_timedelta(time, unit = 's')
    date = date.strftime('%b, %d %Y') + f"\n{time/86400:.1f} days since start of {datestart}"
    for i, hdslayer in enumerate(hds):
        mapview = flopy.plot.PlotMapView(m,ax = axes[i])
        linecollection = mapview.plot_grid(linewidth = .3)  
        # ax.set_title(f"layer {lay+1} elevation (feet)")
        # Hide X and Y axes tick marks
        axes[i].set_xticks([])
        axes[i].set_yticks([])
        # hdslayer[hdslayer==-999] = np.nan
        # im = axes[i].imshow(hdslayer, vmin=0, vmax=75)
        axes[i].set_title("Layer {}".format(i + 1))
        ma = np.ma.array(hdslayer,mask = m.bas6.ibound.array[i]==0)
        quadmesh = mapview.plot_array(ma, vmax = 80,vmin  =20,cmap = 'gist_ncar_r')
        # ctr = axes[i].contour(ma, colors="k", linewidths=0.5, vmin = 0, vmax = 75)

        set_bounds(axes[i], 'mirabel')
        ctx.add_basemap(axes[i], crs = 2226)

        swr_reaches.plot(ax = axes[i], facecolor = "None", edgecolor = 'k')


    # fig.delaxes(axes[-1])
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.03, 0.7])
    fig.colorbar(quadmesh, cax=cbar_ax, label="Head");
    title = f'Heads at {date}'
    fig.suptitle(title)
    filename = f'Day{cnt}.png'
    plt.savefig(os.path.join(out_folder, 'wl_maps',filename), bbox_inches = 'tight')
    

    gal = """<div class="gallery">
      <a target="_blank" href="wl_maps/{:}">
        <img src="wl_maps/{:}" alt="{:}" width="1000" height="1000">
      </a>
      <div class="desc">{:}</div>
    </div>""".format
    f = os.listdir(os.path.join(out_folder,'hydrographs'))

    gallery.append(gal(filename,  filename, title.split('\n')[0], title.split('\n')[0]))
    
with open(os.path.join(out_folder, 'gallery_wl_new.html'), 'w') as outfile:
    outfile.write(head)
    for v in gallery:
        outfile.write(v)
    outfile.write(tail)

In [ ]:



fig, axes = plt.subplots(1, 3, figsize=(20, 8.5), subplot_kw=dict(projection=ccrs.epsg(2226)))
axes = axes.flat
grid = m.modelgrid
for i, hdslayer in enumerate(hds):

    mapview = flopy.plot.PlotMapView(m,ax = axes[i])
    linecollection = mapview.plot_grid(linewidth = .3)

    
    
    # ax.set_title(f"layer {lay+1} elevation (feet)")
    # Hide X and Y axes tick marks
    axes[i].set_xticks([])
    axes[i].set_yticks([])

    
    # hdslayer[hdslayer==-999] = np.nan
    # im = axes[i].imshow(hdslayer, vmin=0, vmax=75)
    axes[i].set_title("Layer {}".format(i + 1))
    ma = np.ma.array(hdslayer,mask = m.bas6.ibound.array[i]==0)
    quadmesh = mapview.plot_array(ma, vmax = 80,vmin  =20,cmap = 'gist_ncar_r')
    # ctr = axes[i].contour(ma, colors="k", linewidths=0.5, vmin = 0, vmax = 75)
    
    set_bounds(axes[i], 'mirabel')
    ctx.add_basemap(axes[i], crs = 2226)
    
    swr_reaches.plot(ax = axes[i], facecolor = "None", edgecolor = 'k')

    # export head rasters
    # (GeoTiff export requires the rasterio package; for ascii grids, just change the extension to *.asc)
    flopy.export.utils.export_array(
        grid, "Output_heads/heads{}.tif".format(i + 1), hdslayer
    )

    # export head contours to a shapefile
    flopy.export.utils.export_array_contours(
        grid, "Output_heads/heads{}.shp".format(i + 1), hdslayer
    )

# fig.delaxes(axes[-1])
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.03, 0.7])
fig.colorbar(quadmesh, cax=cbar_ax, label="Head");
fig.suptitle('Ending Heads')
plt.savefig(os.path.join(out_folder, 'wl_maps', 'Ending Heads.png'), bbox_inches = 'tight')

In [ ]:

get_gradients()

In [ ]:
grad = get_gradients(hds, m, nodata=-999.)

fig, axes = plt.subplots(1,2, figsize=(11, 8.5), subplot_kw=dict(projection=ccrs.epsg(2226)))
axes = axes.flat

for i, vertical_gradient in enumerate(grad):
    # im = axes[i].imshow(vertical_gradient, vmin=grad.min(), vmax=grad.max())
    mapview = flopy.plot.PlotMapView(m,ax = axes[i])
    linecollection = mapview.plot_grid(linewidth = .3)

    quadmesh = mapview.plot_array(vertical_gradient, vmax=grad.max()*1.2,vmin=grad.min()*1.2,cmap = 'gist_ncar_r')
    swr_reaches.plot(ax = axes[i], facecolor = "None", edgecolor = 'k')
    set_bounds(axes[i], 'mirabel')
    ctx.add_basemap(axes[i], crs = 2226)
    
        # Hide X and Y axes tick marks
    axes[i].set_xticks([])
    axes[i].set_yticks([])
    
    axes[i].set_title(
        "Vertical gradient\nbetween Layers {} and {}".format(i + 1, i + 2)
    )
    ctr = axes[i].contour(
        vertical_gradient,
        levels=[-0.1, -0.05, 0.0, 0.05, 0.1],
        colors="k",
        linewidths=0.5,
    )
    plt.clabel(ctr, fontsize=8, inline=1)
    # axes[i].set_bounds()
    

# fig.delaxes(axes[-2])
# fig.delaxes(axes[-1])
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.03, 0.7])
fig.colorbar(quadmesh, cax=cbar_ax, label="positive downward");

plt.savefig(os.path.join('Output_heads', 'Vertical gradient.png'), bbox_inches = 'tight')

In [ ]:

fig, axes = plt.subplots(1,4, figsize=(20, 8.5),  subplot_kw=dict(projection=ccrs.epsg(2226)))
axes = axes.flat
st = m.modelgrid.saturated_thick(hds, mask=-999.0)

st = np.ma.array(st, mask = m.bas6.ibound.array==0)

for lay in range(m.dis.nlay):

    
    mapview = flopy.plot.PlotMapView(m,ax = axes[lay])
    
    # linecollection = mapview.plot_grid(linewidth = .3)

    quadmesh = mapview.plot_array(st[lay], vmin =0, vmax = 80,cmap = 'gist_ncar_r')
    # mapview.plot_ibound(ibound =m.bas6.ibound.array[lay])
    # plt.imshow(st[0])
    # plt.colorbar(label="Saturated thickness")
    axes[lay].set_title(f"Layer {lay+1}");
    set_bounds(axes[lay], 'mirabel')
    swr_reaches.plot(ax = axes[lay], facecolor = "None", edgecolor = 'k')
    ctx.add_basemap(axes[lay], crs = 2226, attribution = 's')
    
    axes[lay].set_xticks([])
    axes[lay].set_yticks([])
    
fig.delaxes(axes[-1])
cbar_ax = fig.add_axes([0.75, 0.35, 0.03, 0.35])
fig.colorbar(quadmesh, cax=cbar_ax, label="Saturated thickness");
fig.suptitle("Saturated Thickness")

plt.savefig(os.path.join('Output_heads', 'Saturated Thickness.png'), bbox_inches = 'tight')


## Get the water table

get_water_table() returns an nrow, ncol array of the water table elevation.
This method can be useful when HDRY is turned on and the water table is in multiple layers.


In [ ]:
fig, axes = plt.subplots(1,1, figsize=(8, 8.5),  subplot_kw=dict(projection=ccrs.epsg(2226)))


wt = get_water_table(heads=hds, nodata=-999.0)
wt = np.ma.array(wt, mask = m.bas6.ibound.array[0]==0)

# wt = np.ma.array(wt, mask = m.bas6.ibound.array==0)

lay = 0

    
mapview = flopy.plot.PlotMapView(m,ax = axes)

# linecollection = mapview.plot_grid(linewidth = .3)

quadmesh = mapview.plot_array(wt, vmin =0, vmax = 80,cmap = 'gist_ncar_r')

set_bounds(axes, 'mirabel')
ctx.add_basemap(axes, crs = 2226, attribution = 's')

axes.set_xticks([])
axes.set_yticks([])

swr_reaches.plot(ax = axes, facecolor = "None", edgecolor = 'k')


cbar_ax = fig.add_axes([0.95, 0.25, 0.03, 0.5])
fig.colorbar(quadmesh, shrink =.15, cax=cbar_ax, label="Watertable Elevation");
fig.suptitle("Watertable Elevation")


In [ ]:
fig, axes = plt.subplots(1,1, figsize=(8, 8.5),  subplot_kw=dict(projection=ccrs.epsg(2226)))


wt = m.bas6.strt.array[0]
wt = np.ma.array(wt, mask = m.bas6.ibound.array[0]==0)

# wt = np.ma.array(wt, mask = m.bas6.ibound.array==0)

lay = 0

    
mapview = flopy.plot.PlotMapView(m,ax = axes)

# linecollection = mapview.plot_grid(linewidth = .3)

quadmesh = mapview.plot_array(wt, vmin =0, vmax = 80,cmap = 'gist_ncar_r')

set_bounds(axes, 'mirabel')
ctx.add_basemap(axes, crs = 2226, attribution = 's')

axes.set_xticks([])
axes.set_yticks([])



cbar_ax = fig.add_axes([0.95, 0.25, 0.03, 0.5])
fig.colorbar(quadmesh, shrink =.15, cax=cbar_ax, label="Starting Heads");
fig.suptitle("Starting Heads")
swr_reaches.plot(ax = axes, facecolor = "None", edgecolor = 'k')
plt.savefig(os.path.join('Output_heads', 'starting_heads.png'), bbox_inches = 'tight')